# Вебинар 4. Домашнее задание

1. Перенесите метрики в модуль metrics.py (убедится что они там)
2. Перенесите функцию prefilter_items в модуль utils.py
3. Создайте модуль recommenders.py. Напищите код для класса ниже 
(задание обсуждали на вебинаре, для первой функции практически сделали) и положите его в recommenders.py
4. Проверьте, что все модули корректно импортируются

In [53]:
import pandas as pd
import numpy as np

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight


class MainRecommender:
    """Рекоммендации, которые можно получить из ALS

    Input
    -----
    user_item_matrix: pd.DataFrame
        Матрица взаимодействий user-item
    """

    def __init__(self, data, weighting=True):

        self.data = data
        self.user_item_matrix = self.prepare_matrix(data)  # pd.DataFrame
        self.id_to_itemid, self.id_to_userid, \
        self.itemid_to_id, self.userid_to_id = self.prepare_dicts(self.user_item_matrix)

        if weighting:
            self.user_item_matrix = bm25_weight(self.user_item_matrix.T).T

        self.model = self.fit(self.user_item_matrix)
        self.own_recommender = self.fit_own_recommender(self.user_item_matrix)
        self.pop_item = 0
        self.pop_list = 0

    @staticmethod
    def prepare_matrix(data):

        user_item_matrix = pd.pivot_table(data,
                                          index='user_id', columns='item_id',
                                          values='quantity',
                                          aggfunc='count',
                                          fill_value=0
                                          )

        user_item_matrix = user_item_matrix.astype(float)

        return user_item_matrix

    @staticmethod
    def prepare_dicts(user_item_matrix):
        """Подготавливает вспомогательные словари"""

        userids = user_item_matrix.index.values
        itemids = user_item_matrix.columns.values

        matrix_userids = np.arange(len(userids))
        matrix_itemids = np.arange(len(itemids))

        id_to_itemid = dict(zip(matrix_itemids, itemids))
        id_to_userid = dict(zip(matrix_userids, userids))

        itemid_to_id = dict(zip(itemids, matrix_itemids))
        userid_to_id = dict(zip(userids, matrix_userids))

        return id_to_itemid, id_to_userid, itemid_to_id, userid_to_id

    @staticmethod
    def fit_own_recommender(user_item_matrix):
        """Обучает модель, которая рекомендует товары, среди товаров, купленных юзером"""

        own_recommender = ItemItemRecommender(K=1, num_threads=4)
        own_recommender.fit(csr_matrix(user_item_matrix).T.tocsr())

        return own_recommender

    def prepare_popularity_matrix(self,  n=5):
        """топ-N покупок пользователя"""

        popularity = self.data.groupby(['user_id', 'item_id'])['quantity'].count().reset_index()
        popularity.sort_values('quantity', ascending=False, inplace=True)
        popularity = popularity[popularity['item_id'] != 999999]

        self.pop_item = popularity.head(1).item_id.values
        self.pop_list = popularity.head(n).item_id.to_list()

        popularity = popularity.groupby('user_id').head(n)
        popularity.sort_values(by=['user_id', 'quantity'], ascending=False, inplace=True)

        return popularity


    @staticmethod
    def fit(user_item_matrix, n_factors=20, regularization=0.001, iterations=15, num_threads=4):
        """Обучает ALS"""

        model = AlternatingLeastSquares(factors=n_factors,
                                        regularization=regularization,
                                        iterations=iterations,
                                        num_threads=num_threads)
        model.fit(csr_matrix(user_item_matrix).T.tocsr())

        return model

    def get_similar_items_recommendation(self, user, N=5):
        """Рекомендуем товары, похожие на топ-N купленных юзером товаров"""

        popularity = self.prepare_popularity_matrix(n=N)
        popular_user_items = popularity.item_id[popularity['user_id'] == user]

        res = []
        for item in popular_user_items:
            recs = self.model.similar_items(self.itemid_to_id[item], N=2)
            top_rec = recs[1][0]
            res.append(self.id_to_itemid[top_rec])

        # обработка пользователей, у которых меньше 5 покупок (оставшиеся товары рекомендуем из популярных)
        i = 0
        while len(res) != N:
            res.append(self.pop_list[i])
            i += 1

        assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res

    def get_similar_users_recommendation(self, user, N=5):
        """Рекомендуем топ-N товаров, среди купленных похожими юзерами"""

        popularity = self.prepare_popularity_matrix(1)
        similar_users = self.model.similar_users(self.userid_to_id[user], N=N+1)

        res = []
        for i in range(1, N+1):
            similar_user = similar_users[i][0]
            popular_item = popularity[popularity['user_id'] == similar_user]['item_id'].values

            if len(popular_item) != 0:
                res.append(int(popular_item))
            else:
                try:
                    res.append(self.pop_item[0])
                except KeyError:
                    print(f'Error user {user}')

        assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res


----

Проверка, что все работает

In [36]:
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.bpr import BayesianPersonalizedRanking
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

In [5]:
data = pd.read_csv('../retail_train.csv')


data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)

# Разбиваем данные на train и test (для test возьмем данные за последние 3 недели)
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [6]:
# Warm start - берем только те user_id, которые есть в train

data_test = data_test[data_test['user_id'].isin(data_train['user_id'])]

In [7]:
# отфильтруем товары с помощью функции prefilter_items

n_items_before = data_train['item_id'].nunique()

data_train = prefilter_items(data_train, take_n_popular=5000)

n_items_after = data_train['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 86865 to 5001


In [8]:
# соберем таблицу с рекомендациями для каждого пользователя для посчета метрик

result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']

result = result.loc[result['user_id'] != 1984]
result = result.loc[result['user_id'] != 2259]

result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [54]:
model = MainRecommender(data_train)

result['similar_users_top'] = result['user_id'].apply(lambda x: model.get_similar_users_recommendation(x, N=5))

result.head(5)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

,user_id,actual,similar_users_top
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1106523, 1106523, 1126899, 10150194, 5570467]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1098284, 8014645, 1053329, 999625, 9553382]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[823862, 9926758, 1110843, 1075368, 1092937]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[921504, 937110, 1029605, 6463717, 944137]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1126899, 937292, 5568489, 951727, 1126899]"


In [56]:
precision= result.apply(lambda x: precision_at_k(x['similar_users_top'], x['actual']), axis=1).mean()
precision

0.04156862745098039

In [57]:
result['user_top_similar_items'] = result['user_id'].apply(lambda x: model.get_similar_items_recommendation(x, N=5))
result.head(5)

,user_id,actual,similar_users_top,user_top_similar_items
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1106523, 1106523, 1126899, 10150194, 5570467]","[824758, 1031087, 990656, 5577022, 9803545]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1098284, 8014645, 1053329, 999625, 9553382]","[1044078, 8090537, 832678, 839419, 7168739]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[823862, 9926758, 1110843, 1075368, 1092937]","[948650, 5569845, 1105488, 8357613, 941361]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[921504, 937110, 1029605, 6463717, 944137]","[1060529, 1044078, 1114435, 949023, 8293385]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1126899, 937292, 5568489, 951727, 1126899]","[5569845, 5569374, 1005186, 1044078, 1044078]"


In [58]:
precision= result.apply(lambda x: precision_at_k(x['user_top_similar_items'], x['actual']), axis=1).mean()
precision

0.05137254901960785